In [44]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools
using StaticArrays

using Profile
using PProf
using ProfileSVG

import Base: show

In [59]:
meanv = datavar(:mean, NormalMeanPrecision{Float64})
precv = datavar(:precision, GammaAB{Float64})

gmpnode = GaussianMeanPrecisionNode(factorisation = SA[ SA[1], SA[2], SA[3] ])

connect!(gmpnode, :mean, meanv, 1)
connect!(gmpnode, :precision, precv, 1)

y = datavar(:y, Float64)

connect!(gmpnode, :value, y, 1)

activate!(gmpnode)

setbelief!(meanv, NormalMeanPrecision(0.0, 0.00001))
setbelief!(precv, GammaAB(0.0001, 0.0001))

In [60]:
realprecision = 10.0
realmean = -200.0

data = rand(Normal(realmean, sqrt(1.0 / realprecision)), (1000, ));

n = 100;
iters = 4;

In [61]:
currentmeanv = nothing
currentprecv = nothing

subscribe!(getbelief(meanv), (d) -> global currentmeanv = getdata(d))
subscribe!(getbelief(precv), (d) -> global currentprecv = getdata(d))

averageE = combineLatest(
    getbelief(meanv), getbelief(precv), getbelief(y), 
    strategy = PushNew()
) |> map(Float64, (beliefs) -> averageEnergy(NormalMeanPrecision, beliefs)) 

differentialE = combineLatest(
    getbelief(meanv), getbelief(precv), 
    strategy = PushNew()
) |> map(Float64, (beliefs) -> reduce(+, map(differentialEntropy, beliefs)))

fe = Matrix{Float64}(undef, n, iters)

for i in 1:n
    tmp1 = currentmeanv
    tmp2 = currentprecv
    
    averageGamma = combineLatest(
        of(as_belief(currentprecv.a)), of(as_belief(currentprecv.b)), getbelief(precv),
        strategy = PushNew()
    ) |> map(Float64, (d) -> averageEnergy(GammaAB, d))
    
    averageNormal = combineLatest(
        of(as_belief(currentmeanv.mean)), of(as_belief(currentmeanv.precision)), getbelief(meanv),
        strategy = PushNew()
    ) |> map(Float64, (d) -> averageEnergy(NormalMeanPrecision, d))
    
    freeEnergy = combineLatest(
        averageE, averageGamma, averageNormal, differentialE,
        strategy = PushNew()
    ) |> map(Float64, (d) -> d[1] + d[2] + d[3] - d[4])

    for j in 1:iters
        subscription = subscribe!(freeEnergy, (d) -> fe[i, j] = d)
        update!(y, data[i])
        update!(meanv, tmp1)
        update!(precv, tmp2)
        unsubscribe!(subscription)
    end
    # update!(meanv, currentmeanv)
    # update!(precv, currentprecv)
end

println("meanv ", mean(currentmeanv))
println("precv ", mean(currentprecv))

meanv -200.04122037405904
precv 10.203264835320187


In [48]:
sum(fe, dims = 1)

1×4 Array{Float64,2}:
 79981.3  2.57  2.5697  2.56969

In [49]:
fe

100×4 Array{Float64,2}:
 79994.1        15.3028     15.3028     15.3028
     1.70722     1.70031     1.70002     1.70001
     0.995344    0.992939    0.99293     0.99293
     0.665403    0.664405    0.664404    0.664404
     0.460498    0.459984    0.459983    0.459983
     0.886127    0.8861      0.8861      0.8861
     0.244327    0.244185    0.244185    0.244185
     0.14837     0.148277    0.148277    0.148277
     0.156757    0.156636    0.156636    0.156636
     0.57577     0.575764    0.575764    0.575764
     0.0394635   0.0394084   0.0394084   0.0394084
     1.01996     1.01919     1.01919     1.01919
     1.06296     1.06233     1.06233     1.06233
     ⋮                                  
    -0.367403   -0.367403   -0.367403   -0.367403
    -0.501625   -0.501625   -0.501625   -0.501625
    -0.630701   -0.630701   -0.630701   -0.630701
    -0.630815   -0.630815   -0.630815   -0.630815
    -0.0486129  -0.048613   -0.048613   -0.048613
    -0.625964   -0.625964   -0.625964   -0

In [9]:
ReactiveMP.averageEnergy(GammaAB, (Belief(1.0), Belief(1.0), Belief(GammaAB(1.0, 1.0))))

1.0